In [2]:
from math import exp
import numpy as np
from gym import spaces
import gym

In [3]:
alpha = [0.5, 0.3, 0.6, 0.2, 0.5, 0.8, 0.2, 0.1]
cap = 0 
cap_max = 70
p_mi = 0 
p_max = 35
p = 0
𝜇= 30
𝜎 = 5
penalty = -50

In [4]:
def cost_func(t, p):
    return alpha[t] * exp(p)

In [35]:
def power_consumed(𝜇= 30, 𝜎 = 5, n=1):
    return np.random.normal(𝜇,𝜎,n)[0]

In [36]:
print(power_consumed())

34.61908072758543


In [24]:
class SmartCharging(gym.Env):
    def __init__(self, actions, cap_max, penalty):
        super(SmartCharging, self).__init__()
        
        self.action_space = spaces.Discrete(actions)
        
        self.observation_space = spaces.Dict({
            'cap': spaces.Discrete(cap_max+1),
            'time': spaces.Discrete(8)
        })
        
        self.state = {
            'cap': 0,
            'time': 0
        }
        
    def reset(self,cap):
        self.state = {
            'cap' : cap,
            'time' : 0
        }
        
    def step(self,action):
        reward = 0
        if action + cap > cap_max:
            
            reward += cost_func(self.state['time'], action + cap - cap_max)
            self.state['cap'] = cap_max
            self.state['time'] += 1
        else
            reward += cost_func(self.state['time'], action + cap - cap_max)
            
            self.state['cap'] = cap_max
            self.state['time'] += 1
        if self.state['time'] == 7:
            
            consumed = power_consumed()
            if self.state['cap'] - consumed > 0:
                self.reset()
            
        return reward

In [25]:
env = SmartCharging(4,75, penalty)

In [26]:
env.state

{'cap': 0, 'time': 0}

In [27]:
env.reset(30)

In [28]:
env.state

{'cap': 30, 'time': 0}